In [1]:
import torch
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
import os
from torch.utils.data.sampler import SubsetRandomSampler
import sklearn
from sklearn.metrics import confusion_matrix
from io import BytesIO
import captum
from captum.attr import IntegratedGradients
from captum.attr import visualization as viz
from captum.attr import NoiseTunnel
from torchsummary import summary
import wget
import sqlite3, json, io
from tqdm import tqdm
import os
import numpy as np
import re
import string
from pprint import pprint
import json
import math
from math import sqrt
import torch
import itertools 
import torchtext
from torchtext.data import Field, Dataset, Example
import sklearn
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from torchtext.data import Iterator, BucketIterator
import torch.nn as nn
from torch.nn import Module
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
import tqdm
import random
import pandas as pd
from collections import Counter
from google.cloud import vision
import sys,os,json
from dotenv import load_dotenv
load_dotenv()
import pymongo
from pymongo import MongoClient
from google.cloud import vision
from google.protobuf.json_format import MessageToJson
import requests
import logging
import googletrans
from googletrans import Translator
from torch.utils.data import DataLoader
import cv2
from PIL import Image
import io
from io import BytesIO
from itertools import chain

In [ ]:
# wget.download("https://dl.fbaipublicfiles.com/fasttext/vectors-aligned/wiki.hi.align.vec")

## Load images and image text 

In [4]:
transformations=transforms.Compose([
                    transforms.Resize((224,224)), 
                    transforms.ToTensor(),
                    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])

class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, txt_path, img_dir, 
                 transform):
        df = pd.read_csv(txt_path, sep=" ", header=None, names= ["img_name"])
        df["label"] = df["img_name"].apply(lambda x: 0 if "neg" in x else 1)
        self.img_dir=img_dir
        self.txt_path=txt_path
        self.img_names=df["img_name"].values
        self.y = df["label"].values
        self.transform=transform
        
    def __getitem__(self, idx):
        img = Image.open(os.path.join(self.img_dir, self.img_names[idx])).convert('RGB')
        client = vision.ImageAnnotatorClient()
        img_text = self.get_image_text(os.path.join(self.img_dir, self.img_names[idx]), client)
        if self.transform is not None:
            img_vector=self.transform(img)
        
        label = self.y[idx]
        return img_vector,label,img_text
    
    def __len__(self):
        return self.y.shape[0]
    
    def get_image_text(self, path, client):  
        def detect_text(img_bytes, client):
            image_data = vision.types.Image(content=img_bytes)
            resp = client.text_detection(image=image_data)
            resp = json.loads(MessageToJson(resp))
            text = resp.get('fullTextAnnotation',{}).get("text","")
            return text
        
        image = Image.open(path, mode="r")
        imgByteArr = io.BytesIO()
        image.save(imgByteArr, format="PNG")
        img_bytes = imgByteArr.getvalue()
        text = detect_text(img_bytes, client)
        return text
    


In [5]:
# dataset = CustomDataset(txt_path="/Users/kruttikanadig/Documents/Tattle/machine-learning/textfile.txt",
#                        img_dir="/Users/kruttikanadig/Documents/Tattle/machine-learning/all",
#                        transform=transformations)

In [231]:
dataset = CustomDataset(txt_path="/Users/kruttikanadig/Documents/Tattle/machine-learning/temp.txt",
                       img_dir="/Users/kruttikanadig/Documents/Tattle/machine-learning/few",
                       transform=transformations)

## Create text preprocessing pipeline

1. Tokenize training data and get word counts
2. Load aligned word embeddings into aligned dict 
2. Create vocab and vocab2index using aligned dict and word counts
3. Encode training data using vocab and vocab2index 
4. Apply to validation / test data 

In [189]:
def prepare_sequences(dataloader, inference=False, path=None, vocab2index=None):
    """ Encodes text into sequences of vocabulary indices """
    
    def tokenize(text):
        """ Removes punctuation and numbers, converts to lowercase and splits into individual words """
        regex = re.compile('[' + re.escape(string.punctuation) + '0-9\\r\\t\\n\।]') # last character । is the Hindi full stop
        nopunct = regex.sub(" ", text.lower())
        tokenized_text = [token for token in nopunct.split(" ") if len(token) > 0]
        return tokenized_text
    
    def count_words(tokens, counts):
        """ Counts unique words in text """
        counts.update(list(chain.from_iterable(tokens)))
        return counts
    
    def load_embeddings(path):
        """ Loads pretrained aligned word embeddings """
        words = []
        idx = 0
        word2idx = {}
        vectors = []
        fin = io.open(path, 'r', encoding='utf-8', newline='\n', errors='ignore')
        n, d = map(int, fin.readline().split())
        for line in tqdm.tqdm(fin,total=n):
            tokens = line.rstrip().split(' ')
            vec = (list(map(float, tokens[1:])))
            word = tokens[0].replace("'",'"')
            words.append(word)
            word2idx[word]=idx
            vectors.append(vec)
            idx+=1

        vectors = np.array(vectors)
        aligned_dict = {w: vectors[word2idx[w]] for w in words}
        return aligned_dict
    
    def get_emb_matrix(pretrained, word_counts, emb_size = 300):
        """ Creates training vocabulary, vocab2index and embedding matrix from pretrained word vectors """
        found=0
        not_found=0
        vocab_size = len(word_counts) + 2
        vocab_to_idx = {}
        vocab = ["", "UNK"]
        W = np.zeros((vocab_size, emb_size), dtype="float32")
        W[0] = np.zeros(emb_size, dtype='float32') # adding a vector for padding
        W[1] = np.random.uniform(-0.25, 0.25, emb_size) # adding a vector for unknown words 
        vocab_to_idx["UNK"] = 1
        i = 2
        for word in word_counts:
            if word in pretrained:
                W[i] = pretrained[word]
                found+=1
            else:
                W[i] = np.random.uniform(-0.25,0.25, emb_size)
                not_found+=1
            vocab_to_idx[word] = i
            vocab.append(word)
            i += 1   
        return W, np.array(vocab), vocab_to_idx, found, not_found
    
    def encode_tokens(tokenized_text, vocab2index, N=50): # keep max doc length to 50 words
        """ Encodes tokenized text into equal length sequences of vocabulary indices """ 
        sequences = []
        for text in tokenized_text:
            seq = np.zeros(N, dtype=int)
            # get word index if it's in vocab else get default index
            enc = np.array([vocab2index.get(word, vocab2index["UNK"]) for word in text])
            # limit sequence length
            length = min(N, len(enc))
            seq[:length] = enc[:length]
            sequences.append(seq)
        return sequences

    if inference==False:
        counts = Counter()
        tokenized_text = []
        print("Tokenizing text and counting unique words")
        for i in dataloader:
            tokenized = [tokenize(doc) for doc in i[2]]
            counts = count_words(tokenized, counts)
            tokenized_text.extend(tokenized)

        print("Loading pretrained aligned word embeddings")
        aligned_dict = load_embeddings(path)

        print("Creating vocab2index and embedding matrix with pretrained weights")
        pretrained_weights, vocab, vocab2index, found, not_found = get_emb_matrix(aligned_dict, counts)
        sequences = encode_tokens(tokenized_text, vocab2index)
        vocab_size = len(vocab)
        print("Finished preparing sequences")
        return sequences, vocab, vocab_size, vocab2index, pretrained_weights
    else:
        print("Tokenizing text")
        tokenized_text = []
        for i in dataloader:
            tokenized = [tokenize(doc) for doc in i[2]]
            tokenized_text.extend(tokenized)
        sequences = encode_tokens(tokenized_text, vocab2index)
        print("Finished preparing sequences")
        return sequences

## Train-validation split

In [242]:
# Shuffle and randomly split dataset into 80:20 training and validation sets
train_size = int(0.9 * len(dataset))
valid_size = len(dataset) - train_size
train_dataset, valid_dataset = torch.utils.data.random_split(dataset, [train_size, valid_size])

In [243]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=2, shuffle=False)
validation_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=2, shuffle=False)

In [247]:
len(dataset)
for i in dataset:
    print(i)

(tensor([[[-2.0323, -2.0323, -2.0323,  ..., -2.0152, -2.0152, -2.0323],
         [-2.0323, -2.0323, -2.0323,  ..., -1.9467, -1.9638, -2.0323],
         [-2.0323, -2.0323, -2.0323,  ..., -1.9124, -1.9124, -1.9980],
         ...,
         [-1.5870, -1.5870, -1.5870,  ..., -1.3644, -1.3473, -1.3473],
         [-1.5699, -1.5699, -1.5699,  ..., -1.3644, -1.3644, -1.3815],
         [-1.5357, -1.5357, -1.5357,  ..., -1.3644, -1.3815, -1.4158]],

        [[-0.6877, -0.6877, -0.6877,  ..., -0.7052, -0.6877, -0.6702],
         [-0.6877, -0.6877, -0.6877,  ..., -0.6702, -0.6702, -0.6702],
         [-0.6877, -0.6877, -0.6877,  ..., -0.7227, -0.6527, -0.6352],
         ...,
         [-0.7402, -0.7402, -0.7402,  ..., -0.3550, -0.3375, -0.3375],
         [-0.7227, -0.7227, -0.7227,  ..., -0.3550, -0.3550, -0.3725],
         [-0.6877, -0.6877, -0.6877,  ..., -0.3550, -0.3725, -0.4076]],

        [[ 0.8274,  0.8274,  0.8274,  ...,  0.7054,  0.7228,  0.7576],
         [ 0.8274,  0.8274,  0.8274,  ...,  

In [222]:
path = "/Users/kruttikanadig/Documents/Tattle/machine-learning/wiki.hi.align.vec"
train_sequences, vocab, vocab_size, vocab2index, pretrained_weights = prepare_sequences(train_loader, inference=False, path=path)

Tokenizing text and counting unique words


  1%|▏         | 1979/158016 [00:00<00:15, 9922.27it/s]

Loading pretrained aligned word embeddings


100%|██████████| 158016/158016 [00:19<00:00, 8131.42it/s] 


Creating vocab2index and embedding matrix with pretrained weights
Finished preparing sequences


In [213]:
valid_sequences = prepare_sequences(validation_loader, inference=True, vocab2index=vocab2index)

Tokenizing text
Finished preparing sequences


In [223]:
train_seq_loader = torch.utils.data.DataLoader(train_sequences, batch_size=2, shuffle=False)
valid_seq_loader = torch.utils.data.DataLoader(valid_sequences, batch_size=2, shuffle=False)

In [187]:
for i, data in enumerate(zip(validation_loader, valid_seq_loader)):
    print(data)
    break

([tensor([[[[ 2.2489,  2.2489,  2.2489,  ...,  2.2147,  2.2489,  2.2489],
          [ 2.2489,  2.2489,  2.2489,  ...,  2.2147,  2.2489,  2.2489],
          [ 2.2489,  2.2489,  2.2489,  ...,  2.2147,  2.2489,  2.2489],
          ...,
          [ 1.3927,  1.3413,  1.3413,  ...,  1.3755,  1.3755,  1.3755],
          [ 1.3584,  1.3584,  1.3584,  ...,  1.3584,  1.3584,  1.3584],
          [ 1.3584,  1.3584,  1.3584,  ...,  1.3584,  1.3584,  1.3584]],

         [[ 2.0959,  2.0959,  2.0959,  ...,  2.0609,  2.0959,  2.0959],
          [ 2.0959,  2.0959,  2.0959,  ...,  2.0609,  2.0959,  2.0959],
          [ 2.0959,  2.0959,  2.0959,  ...,  2.0609,  2.0959,  2.0959],
          ...,
          [ 1.5532,  1.5007,  1.5007,  ...,  1.5357,  1.5357,  1.5357],
          [ 1.5182,  1.5182,  1.5182,  ...,  1.5182,  1.5182,  1.5182],
          [ 1.5182,  1.5182,  1.5182,  ...,  1.5182,  1.5182,  1.5182]],

         [[-0.7936, -0.7936, -0.7936,  ..., -0.8284, -0.7936, -0.7936],
          [-0.7936, -0.7936,

In [13]:
# for k, v in vocab2index.items():
#     if v==102:
#         print (k)

# vocab2index["कर"]

In [224]:
class FusionModel(torch.nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, pretrained_weights, num_labels):
        super().__init__()
        # LSTM layers
        self.embeddings = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.embeddings.weight.data.copy_(torch.from_numpy(pretrained_weights))
        self.embeddings.weight.requires_grad = False ## freeze embeddings
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.linear = nn.Linear(hidden_dim, 100)
        #self.dropout = nn.Dropout(0.2)
        # ResNet layers
        self.resnet = models.resnet18(pretrained=True)
        for param in self.resnet.parameters():
            param.requires_grad = False
        # Replace the default ResNet18 classifier layer 
        classifier_input = self.resnet.fc.in_features
        classifier = nn.Sequential(nn.Linear(classifier_input, 100), 
                             nn.LogSoftmax(dim=1)) 
        self.resnet.fc = classifier
        self.fusion_classifier = nn.Linear(200,2)
        
    def forward(self, x1, x2): # image vector, encoded image text
        print("x1:",x1)
        print("x2",x2)
        # Image
        x1_out = self.resnet(x1)
        # Text
        x2_embed = self.embeddings(x2)
        #x2_dropout = self.dropout(x2_embed)
        print("img representation:",x1_out)
        print("")
        lstm_out, (ht, ct) = self.lstm(x2_embed)
        x2_out = self.linear(ht[-1])
        print("text representation:",x2_out)
        print("")
        # Concatenation
        fused = torch.cat([x2_out, x1_out], dim=1)
        print("fused:", fused)

        return self.fusion_classifier(fused)

In [225]:
model = FusionModel(vocab_size=vocab_size, embedding_dim=300, hidden_dim=5, pretrained_weights=pretrained_weights,
                    num_labels=2)


In [216]:

def train_model(model, epochs, lr, train_loader, train_seq_loader):
    parameters = filter(lambda p: p.requires_grad, model.parameters())
    optimizer = torch.optim.Adam(parameters, lr=lr)
    model.train()
    for i in range(epochs):
        sum_loss = 0.0
        total = 0
        for i, data in enumerate(zip(train_loader, train_seq_loader)):
        #for data in zip(validation_loader, valid_seq_loader): 
            img_vectors, text_sequences, y_train = data[0][0], data[1], data[0][1]      
#             x = x.long()
#             y = y.long()
            optimizer.zero_grad()
            y_pred = model(img_vectors, text_sequences)
            loss = F.cross_entropy(y_pred, y_train)
            loss.backward()
            optimizer.step()
            sum_loss += loss.item()*y_train.shape[0]
            total += y_train.shape[0]
        val_loss, val_acc = validation_metrics(model, validation_loader, valid_seq_loader)
        print("train loss %.3f, val loss %.3f, val accuracy %.3f" % (sum_loss/total, val_loss, val_acc))

def validation_metrics (model, validation_loader=validation_loader, valid_seq_loader=valid_seq_loader):
    model.eval()
    correct = 0
    total = 0
    sum_loss = 0.0
    for i, data in enumerate(zip(validation_loader, valid_seq_loader)):
        #
        img_vectors, text_sequences, y_valid = data[0][0], data[1], data[0][1] 
#         x = x.long()
#         y = y.long()
        y_hat = model(img_vectors, text_sequences)
        loss = F.cross_entropy(y_hat, y_valid)
        print("y_hat:",y_hat)
        pred = torch.max(y_hat, 1)[1]
        print("pred:",pred)
        correct += (pred == y_valid).float().sum()
        total += y_valid.shape[0]
        sum_loss += loss.item()*y_valid.shape[0]
    return sum_loss/total, correct/total

In [173]:
%%time
train_model(model=model, epochs=10, lr=0.01, train_loader=train_loader, train_seq_loader=train_seq_loader)

tensor([[-1.2884, -1.8953],
        [-1.1243, -1.7167],
        [-1.3596, -1.9504],
        [-0.7396, -1.2463],
        [-0.9927, -1.5336],
        [-1.1708, -1.7369],
        [-1.0991, -1.6546],
        [-0.9892, -1.5298]], grad_fn=<AddmmBackward>)
tensor([0, 0, 0, 0, 0, 0, 0, 0])
tensor([[1.4987, 2.9468],
        [1.0020, 2.0537],
        [1.0462, 2.1332],
        [1.3739, 2.7223],
        [1.4643, 2.8850],
        [0.9805, 2.0152],
        [1.5569, 3.0513],
        [1.6277, 3.1998]], grad_fn=<AddmmBackward>)
tensor([1, 1, 1, 1, 1, 1, 1, 1])
tensor([[-1.4104,  1.8417],
        [-2.2062,  2.5288],
        [-2.1078,  2.4641],
        [-1.9090,  2.3783],
        [-3.5514,  3.4589],
        [-1.8618,  2.1805],
        [-2.8620,  2.9453],
        [-1.9153,  2.2209]], grad_fn=<AddmmBackward>)
tensor([1, 1, 1, 1, 1, 1, 1, 1])
tensor([[-0.7397,  3.7871],
        [-0.8005,  4.2924],
        [-0.7056,  3.8461],
        [-0.6546,  3.0524],
        [-0.8033,  4.3144],
        [-0.6357,  3.3473],

KeyboardInterrupt: 

## Debugging

In [204]:
%%time
train_model(model=model, epochs=2, lr=0.01, train_loader=train_loader, train_seq_loader=train_seq_loader)

img representation: tensor([[-4.2521, -4.2464, -4.4741, -4.5547, -4.5478, -5.5457, -5.3049, -5.1204,
         -5.1457, -5.2056, -4.5313, -5.0253, -4.7385, -4.7661, -4.5265, -4.2586,
         -3.3594, -4.3817, -4.1012, -5.8242, -5.5435, -5.0205, -5.0595, -4.6622,
         -4.5196, -4.2702, -4.5995, -4.6876, -5.2176, -3.2149, -4.6365, -4.9898,
         -5.5991, -4.2091, -4.7610, -4.1638, -3.8122, -4.7662, -5.0462, -4.9526,
         -4.7008, -4.5671, -4.3898, -4.1583, -4.2191, -3.2409, -4.4813, -4.4752,
         -2.9030, -4.5563, -4.8537, -4.8357, -4.2699, -4.6099, -5.8208, -5.1968,
         -4.3736, -4.7389, -5.1744, -4.9667, -3.9274, -5.3632, -4.8872, -5.0379,
         -5.2167, -5.8661, -4.7592, -5.0030, -5.5042, -4.4636, -4.7795, -5.1121,
         -4.3879, -4.8093, -5.2627, -5.0186, -5.2898, -4.6608, -4.6598, -5.9072,
         -5.5910, -4.7094, -4.7134, -5.2310, -5.1905, -4.3843, -4.2540, -4.8409,
         -4.6072, -4.9843, -5.3077, -4.9965, -4.7092, -5.1058, -4.5060, -6.0548,
        

img representation: tensor([[-11.8634,  -3.5502, -12.5868, -11.5374, -11.5819,  -4.5397,  -3.7741,
         -12.0026, -12.0417, -12.6410, -11.9186, -12.4750, -12.8020,  -4.2791,
         -11.7074, -11.7172, -10.8717,  -3.9249, -11.0223,  -5.0255,  -4.0409,
         -12.3385,  -4.9010, -11.6947,  -3.9779, -10.9718, -11.3072, -11.6124,
          -4.0879, -10.6153, -11.6457,  -4.1957,  -4.7426, -12.2339,  -4.7658,
          -3.5958, -11.3617, -11.6555,  -3.8889,  -4.5319, -12.5150,  -4.3914,
          -4.3513,  -3.6762, -10.7070, -10.1089,  -3.0840, -10.9793, -10.0362,
         -11.9951,  -4.2718, -12.1493,  -4.2256,  -3.4949,  -3.9752, -11.8257,
          -3.2105, -11.7752,  -4.6601,  -4.5970,  -2.3097,  -4.9880, -11.9044,
          -4.8332, -12.7361,  -5.1199, -11.5616,  -4.3642, -12.2760, -11.3185,
          -3.7257, -12.6843,  -3.3394,  -4.4739, -13.2273,  -5.1117, -12.5062,
          -4.4007,  -4.1046, -11.7557, -11.9558,  -4.0044, -11.1070, -12.5791,
          -4.1102,  -4.4747,  -3

In [ ]:
a=torch.Tensor(([[-4.2521, -4.2464, -4.4741, -4.5547, -4.5478, -5.5457, -5.3049, -5.1204,
         -5.1457, -5.2056, -4.5313, -5.0253, -4.7385, -4.7661, -4.5265, -4.2586,
         -3.3594, -4.3817, -4.1012, -5.8242, -5.5435, -5.0205, -5.0595, -4.6622,
         -4.5196, -4.2702, -4.5995, -4.6876, -5.2176, -3.2149, -4.6365, -4.9898,
         -5.5991, -4.2091, -4.7610, -4.1638, -3.8122, -4.7662, -5.0462, -4.9526,
         -4.7008, -4.5671, -4.3898, -4.1583, -4.2191, -3.2409, -4.4813, -4.4752,
         -2.9030, -4.5563, -4.8537, -4.8357, -4.2699, -4.6099, -5.8208, -5.1968,
         -4.3736, -4.7389, -5.1744, -4.9667, -3.9274, -5.3632, -4.8872, -5.0379,
         -5.2167, -5.8661, -4.7592, -5.0030, -5.5042, -4.4636, -4.7795, -5.1121,
         -4.3879, -4.8093, -5.2627, -5.0186, -5.2898, -4.6608, -4.6598, -5.9072,
         -5.5910, -4.7094, -4.7134, -5.2310, -5.1905, -4.3843, -4.2540, -4.8409,
         -4.6072, -4.9843, -5.3077, -4.9965, -4.7092, -5.1058, -4.5060, -6.0548,
         -5.5167, -5.2616, -5.4040, -5.0175]])
               
b=torch.Tensor(([[-11.9913,  -3.7826, -12.5833, -11.7889, -11.2992,  -4.7991,  -4.4945,
         -12.6855, -11.9649, -12.2276, -11.8422, -11.8447, -12.0405,  -3.7416,
         -11.5485, -11.9999, -10.8261,  -3.6566, -12.4659,  -5.4574,  -3.5221,
         -13.0376,  -4.1919, -12.3176,  -4.1105, -12.0193, -11.7328, -11.7507,
          -4.4628, -11.2013, -12.0474,  -4.1130,  -5.0844, -11.2223,  -3.6835,
          -3.1347, -11.4154, -12.1186,  -4.2668,  -4.5831, -11.7406,  -4.6845,
          -2.9980,  -3.2289, -12.1989, -10.6418,  -4.1819, -11.4468, -10.9723,
         -11.6317,  -4.1813, -12.5192,  -3.4268,  -3.2935,  -5.4684, -12.4832,
          -3.8902, -11.5476,  -4.6908,  -4.4863,  -3.9324,  -4.3077, -12.5412,
          -3.7257, -12.4882,  -4.7619, -12.2484,  -4.2403, -12.7177, -11.5256,
          -4.2174, -12.0858,  -3.4705,  -3.7659, -12.0408,  -3.4726, -12.9526,
          -3.7892,  -4.5419, -12.8421, -12.8729,  -3.7096, -11.8118, -12.6065,
          -4.6220,  -3.4215,  -3.1535,  -4.5739,  -4.1453,  -4.8709, -12.8969,
          -4.0048,  -3.8581,  -4.7366,  -3.8848, -12.9636,  -4.4669,  -5.0619,
          -5.0294,  -3.8282],
        [-11.8447,  -3.9464, -11.2609, -12.2781, -12.1543,  -3.9545,  -4.0019,
         -12.2932, -11.7808, -12.0304, -13.2854, -12.1163, -12.5795,  -3.8014,
         -12.4147, -12.2848, -11.4008,  -3.2728, -11.7993,  -4.4218,  -5.2945,
         -12.2411,  -3.8866, -12.8508,  -4.0057, -11.6302, -11.6989, -11.8493,
          -5.2403,  -9.8713, -11.6305,  -4.2098,  -4.4167, -10.9850,  -3.9780,
          -3.3411, -10.9573, -12.2791,  -3.7425,  -4.7395, -11.8754,  -4.2133,
          -3.4015,  -3.7881, -12.2334, -10.7329,  -3.8347, -11.5902, -11.0319,
         -12.3321,  -3.6539, -12.1957,  -4.0869,  -3.7605,  -5.3315, -12.1294,
          -4.1113, -11.4262,  -4.1386,  -4.7603,  -3.1940,  -5.0062, -11.8343,
          -4.7427, -13.0524,  -5.2751, -12.5379,  -3.4173, -13.7172, -11.6907,
          -3.8422, -12.7018,  -4.2619,  -4.5733, -12.8526,  -4.5814, -12.3846,
          -4.2394,  -4.2135, -13.3160, -14.1304,  -4.3743, -11.3125, -12.6819,
          -5.7403,  -2.4686,  -4.4718,  -3.4121,  -4.4958,  -3.9187, -12.9499,
          -4.2822,  -3.5968,  -4.5703,  -3.0550, -13.7008,  -4.8083,  -3.8884,
          -4.3756,  -4.7645]]))

In [240]:
for i in train_seq_loader:
    print(i)

tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0]])


In [226]:
for i, data in enumerate(zip(train_loader, train_seq_loader)):
        #for data in zip(validation_loader, valid_seq_loader): 
    img_vectors, text_sequences, y_train = data[0][0], data[1], data[0][1]      
#             x = x.long()
#             y = y.long()
    #optimizer.zero_grad()
    y_pred = model(img_vectors, text_sequences)

x1: tensor([[[[-2.0323, -2.0323, -2.0323,  ..., -2.0152, -2.0152, -2.0323],
          [-2.0323, -2.0323, -2.0323,  ..., -1.9467, -1.9638, -2.0323],
          [-2.0323, -2.0323, -2.0323,  ..., -1.9124, -1.9124, -1.9980],
          ...,
          [-1.5870, -1.5870, -1.5870,  ..., -1.3644, -1.3473, -1.3473],
          [-1.5699, -1.5699, -1.5699,  ..., -1.3644, -1.3644, -1.3815],
          [-1.5357, -1.5357, -1.5357,  ..., -1.3644, -1.3815, -1.4158]],

         [[-0.6877, -0.6877, -0.6877,  ..., -0.7052, -0.6877, -0.6702],
          [-0.6877, -0.6877, -0.6877,  ..., -0.6702, -0.6702, -0.6702],
          [-0.6877, -0.6877, -0.6877,  ..., -0.7227, -0.6527, -0.6352],
          ...,
          [-0.7402, -0.7402, -0.7402,  ..., -0.3550, -0.3375, -0.3375],
          [-0.7227, -0.7227, -0.7227,  ..., -0.3550, -0.3550, -0.3725],
          [-0.6877, -0.6877, -0.6877,  ..., -0.3550, -0.3725, -0.4076]],

         [[ 0.8274,  0.8274,  0.8274,  ...,  0.7054,  0.7228,  0.7576],
          [ 0.8274,  0.827

In [230]:
len(train_loader)

1